In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/siamese/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 128
epochs = 4000
start_epoch = 0
train_batch = 32
test_batch = 64
lr = 0.01
schedule = [500, 1000, 2000, 3000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/siamese' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = []

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

# Dataset

In [9]:
train_dir = os.path.join(target_dir, '100_shot_style1')
source_train_dir = os.path.join(target_dir, '100_shot_style1_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])

val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_ = SiameseNetworkDataset(datasets.ImageFolder(val_target_dir), transform=val_aug, should_invert=False)
val_target_loader = DataLoader(val_target_, shuffle=True, num_workers=num_workers, batch_size=test_batch, pin_memory=True)
val_source_ = SiameseNetworkDataset(datasets.ImageFolder(val_source_dir), transform=val_aug, should_invert=False)
val_source_loader = DataLoader(val_source_, batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                                      override_params={'dropout_rate':0.0})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                                      override_params={'dropout_rate':0.0, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/siamese/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 4.17M


In [12]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [13]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [14]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [15]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += torch.norm(param, p=1)
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += torch.norm(param - teacher_model_weights[name], p=1)
    return sp_loss

# Loss

In [16]:
criterion = ContrastiveLoss(margin=1.0).cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [17]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [18]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

        
        outputs0 = student_model(inputs0)
        outputs1 = student_model(inputs1)
        
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            target_index = targets[targets==0]
            target_index = target_index.long().cuda()
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs0.size(), lam)
            inputs0[target_index, :, bbx1:bbx2, bby1:bby2] = inputs1[target_index, :, bbx1:bbx2, bby1:bby2]
        
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)
        loss_main = criterion(outputs0, outputs1, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
    print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{total:.4f} | SPLoss:{eta:.4f} | | CLLoss:{sp:.4f} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader),loss=loss, total=loss_main, eta=loss_sp, sp=loss_cls, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [19]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    with torch.no_grad():
        for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

            # compute output
            outputs0 = model(inputs0)
            outputs1 = model(inputs1)
            outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

            loss_main = criterion(outputs0, outputs1, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            losses.update(loss.data.tolist(), inputs0.size(0))
            top1.update(prec1[0], inputs0.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            # plot progress
            bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                        batch=batch_idx + 1,
                        size=len(val_loader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        total=bar.elapsed_td,
                        eta=bar.eta_td,
                        loss=losses.avg,
                        top1=top1.avg,)
            bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [20]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [21]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
#     if epoch in iter_time:
#         print("iterative training: feedback {}".format(epoch))
#         teacher_model.load_state_dict(student_model.state_dict())
#         teacher_model_weights = {}
#         for name, param in teacher_model.named_parameters():
#             teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.010000
13/13 | Loss:0.3652 | MainLoss:0.3652 | SPLoss:61.6858 | | CLLoss:2883.6365 | top1:46.614585876464844
502/502 Data:0.002 | Batch:0.690 | Total:0:00:58 | ETA:0:00:00 | Loss:0.48188913614579076 | top1:49.99688720703125
122/122 Data:0.002 | Batch:0.755 | Total:0:00:15 | ETA:0:00:00 | Loss:0.015287569312808605 | top1:49.42307662963867

Epoch: [2 | 4000] LR: 0.010600
13/13 | Loss:0.2653 | MainLoss:0.2653 | SPLoss:161.1843 | | CLLoss:2878.5713 | top1:46.35416793823242

Epoch: [3 | 4000] LR: 0.011200
13/13 | Loss:0.3182 | MainLoss:0.3182 | SPLoss:232.7734 | | CLLoss:2875.1982 | top1:54.16666793823242

Epoch: [4 | 4000] LR: 0.011800
13/13 | Loss:0.2796 | MainLoss:0.2796 | SPLoss:291.3589 | | CLLoss:2873.4912 | top1:48.4375

Epoch: [5 | 4000] LR: 0.012400
13/13 | Loss:0.3641 | MainLoss:0.3641 | SPLoss:337.8032 | | CLLoss:2874.0801 | top1:49.739585876464844

Epoch: [6 | 4000] LR: 0.013000
13/13 | Loss:0.1870 | MainLoss:0.1870 | SPLoss:399.0458 | | CLLoss:2874.8069

13/13 | Loss:0.0020 | MainLoss:0.0020 | SPLoss:2952.8867 | | CLLoss:2884.3831 | top1:50.520835876464844

Epoch: [61 | 4000] LR: 0.040000
13/13 | Loss:0.0063 | MainLoss:0.0063 | SPLoss:2966.3779 | | CLLoss:2884.0154 | top1:52.083335876464844

Epoch: [62 | 4000] LR: 0.039999
13/13 | Loss:0.0025 | MainLoss:0.0025 | SPLoss:2980.0439 | | CLLoss:2883.8777 | top1:47.91666793823242

Epoch: [63 | 4000] LR: 0.039999
13/13 | Loss:0.0050 | MainLoss:0.0050 | SPLoss:2989.5608 | | CLLoss:2883.6887 | top1:53.645835876464844

Epoch: [64 | 4000] LR: 0.039999
13/13 | Loss:0.0018 | MainLoss:0.0018 | SPLoss:2999.7358 | | CLLoss:2884.1631 | top1:47.13541793823242

Epoch: [65 | 4000] LR: 0.039999
13/13 | Loss:0.0026 | MainLoss:0.0026 | SPLoss:3007.1670 | | CLLoss:2884.7000 | top1:48.4375

Epoch: [66 | 4000] LR: 0.039999
13/13 | Loss:0.0016 | MainLoss:0.0016 | SPLoss:3011.8579 | | CLLoss:2884.5569 | top1:48.4375

Epoch: [67 | 4000] LR: 0.039999
13/13 | Loss:0.0035 | MainLoss:0.0035 | SPLoss:3023.7693 | | CLLo

13/13 | Loss:0.0005 | MainLoss:0.0005 | SPLoss:3339.1252 | | CLLoss:2881.3953 | top1:48.69791793823242

Epoch: [122 | 4000] LR: 0.039970
13/13 | Loss:0.0017 | MainLoss:0.0017 | SPLoss:3340.3252 | | CLLoss:2881.2996 | top1:50.26041793823242

Epoch: [123 | 4000] LR: 0.039969
13/13 | Loss:0.0030 | MainLoss:0.0030 | SPLoss:3344.6619 | | CLLoss:2881.0349 | top1:53.38541793823242

Epoch: [124 | 4000] LR: 0.039968
13/13 | Loss:0.0027 | MainLoss:0.0027 | SPLoss:3350.4333 | | CLLoss:2880.5322 | top1:51.04166793823242

Epoch: [125 | 4000] LR: 0.039967
13/13 | Loss:0.0008 | MainLoss:0.0008 | SPLoss:3352.1594 | | CLLoss:2880.2307 | top1:47.395835876464844

Epoch: [126 | 4000] LR: 0.039966
13/13 | Loss:0.0008 | MainLoss:0.0008 | SPLoss:3352.3018 | | CLLoss:2880.1011 | top1:47.65625

Epoch: [127 | 4000] LR: 0.039965
13/13 | Loss:0.0025 | MainLoss:0.0025 | SPLoss:3354.9402 | | CLLoss:2879.8247 | top1:55.208335876464844

Epoch: [128 | 4000] LR: 0.039964
13/13 | Loss:0.0011 | MainLoss:0.0011 | SPLoss:3

13/13 | Loss:0.0026 | MainLoss:0.0026 | SPLoss:3569.2815 | | CLLoss:2878.5400 | top1:51.302085876464844

Epoch: [181 | 4000] LR: 0.039897
13/13 | Loss:0.0118 | MainLoss:0.0118 | SPLoss:3571.0005 | | CLLoss:2878.4744 | top1:50.520835876464844

Epoch: [182 | 4000] LR: 0.039896
13/13 | Loss:0.0008 | MainLoss:0.0008 | SPLoss:3576.9424 | | CLLoss:2878.2976 | top1:44.270835876464844

Epoch: [183 | 4000] LR: 0.039894
13/13 | Loss:0.0007 | MainLoss:0.0007 | SPLoss:3580.7485 | | CLLoss:2878.4509 | top1:53.645835876464844

Epoch: [184 | 4000] LR: 0.039893
13/13 | Loss:0.0024 | MainLoss:0.0024 | SPLoss:3582.2522 | | CLLoss:2878.3655 | top1:53.38541793823242

Epoch: [185 | 4000] LR: 0.039891
13/13 | Loss:0.0010 | MainLoss:0.0010 | SPLoss:3582.6433 | | CLLoss:2878.1450 | top1:48.958335876464844

Epoch: [186 | 4000] LR: 0.039889
13/13 | Loss:0.0006 | MainLoss:0.0006 | SPLoss:3585.4087 | | CLLoss:2878.1843 | top1:48.4375

Epoch: [187 | 4000] LR: 0.039888
13/13 | Loss:0.0003 | MainLoss:0.0003 | SPLoss

13/13 | Loss:0.0005 | MainLoss:0.0005 | SPLoss:3699.0815 | | CLLoss:2877.2458 | top1:52.083335876464844

Epoch: [240 | 4000] LR: 0.039782
13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3700.3584 | | CLLoss:2877.2837 | top1:48.4375

Epoch: [241 | 4000] LR: 0.039780
13/13 | Loss:0.0005 | MainLoss:0.0005 | SPLoss:3703.0396 | | CLLoss:2877.4895 | top1:54.427085876464844

Epoch: [242 | 4000] LR: 0.039778
13/13 | Loss:0.0003 | MainLoss:0.0003 | SPLoss:3706.2063 | | CLLoss:2877.6760 | top1:48.4375

Epoch: [243 | 4000] LR: 0.039775
13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3709.0813 | | CLLoss:2877.8853 | top1:48.958335876464844

Epoch: [244 | 4000] LR: 0.039773
13/13 | Loss:0.0010 | MainLoss:0.0010 | SPLoss:3710.4321 | | CLLoss:2877.9045 | top1:45.57291793823242

Epoch: [245 | 4000] LR: 0.039771
13/13 | Loss:0.0017 | MainLoss:0.0017 | SPLoss:3710.5503 | | CLLoss:2877.7764 | top1:52.083335876464844

Epoch: [246 | 4000] LR: 0.039768
13/13 | Loss:0.0003 | MainLoss:0.0003 | SPLoss:3710.6084 

...

13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:54.16666793823242

Epoch: [3738 | 4000] LR: 0.000000
13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:51.04166793823242

Epoch: [3739 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:46.875

Epoch: [3740 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:48.4375

Epoch: [3741 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:51.302085876464844

Epoch: [3742 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:48.4375

Epoch: [3743 | 4000] LR: 0.000000
13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:46.09375

Epoch: [3744 | 4000] LR: 0.000000
13/13 | Loss:0.0000 | MainLoss:0.0000 | SPLoss:3995.0088 | | CLLoss:2871.

13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:52.864585876464844

Epoch: [3797 | 4000] LR: 0.000000
13/13 | Loss:0.0003 | MainLoss:0.0003 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:50.0

Epoch: [3798 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:50.520835876464844

Epoch: [3799 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:44.270835876464844

Epoch: [3800 | 4000] LR: 0.000000
13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:46.875

Epoch: [3801 | 4000] LR: 0.000000
13/13 | Loss:0.0008 | MainLoss:0.0008 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:45.57291793823242
502/502 Data:0.002 | Batch:0.078 | Total:0:00:56 | ETA:0:00:00 | Loss:0.38666598215281406 | top1:49.56074905395508
122/122 Data:0.002 | Batch:0.102 | Total:0:00:14 | ETA:0:00:00 | Loss:0.10681598814634176 | top1:50.2692298889160

13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:49.21875

Epoch: [3854 | 4000] LR: 0.000000
13/13 | Loss:0.0003 | MainLoss:0.0003 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:50.26041793823242

Epoch: [3855 | 4000] LR: 0.000000
13/13 | Loss:0.0003 | MainLoss:0.0003 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:47.91666793823242

Epoch: [3856 | 4000] LR: 0.000000
13/13 | Loss:0.0004 | MainLoss:0.0004 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:49.21875

Epoch: [3857 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:53.125

Epoch: [3858 | 4000] LR: 0.000000
13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:49.47916793823242

Epoch: [3859 | 4000] LR: 0.000000
13/13 | Loss:0.0006 | MainLoss:0.0006 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:46.614585876464844

Epoch: [3860 | 4000] LR: 0.000000
13/13 | Loss:0.0003 | MainLoss:0.0003 | SPLoss:3995.0088 | | C

13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:51.302085876464844

Epoch: [3914 | 4000] LR: 0.000000
13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:45.3125

Epoch: [3915 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:45.57291793823242

Epoch: [3916 | 4000] LR: 0.000000
13/13 | Loss:0.0005 | MainLoss:0.0005 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:49.21875

Epoch: [3917 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:48.69791793823242

Epoch: [3918 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:49.21875

Epoch: [3919 | 4000] LR: 0.000000
13/13 | Loss:0.0003 | MainLoss:0.0003 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:48.69791793823242

Epoch: [3920 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | 

13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:51.04166793823242

Epoch: [3974 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:54.427085876464844

Epoch: [3975 | 4000] LR: 0.000000
13/13 | Loss:0.0005 | MainLoss:0.0005 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:54.427085876464844

Epoch: [3976 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:52.34375

Epoch: [3977 | 4000] LR: 0.000000
13/13 | Loss:0.0002 | MainLoss:0.0002 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:52.083335876464844

Epoch: [3978 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:52.34375

Epoch: [3979 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:3995.0088 | | CLLoss:2871.2615 | top1:47.91666793823242

Epoch: [3980 | 4000] LR: 0.000000
13/13 | Loss:0.0001 | MainLoss:0.0001 | SPLoss:39